In [1]:
# This mounts your Google Drive to the Colab VM.
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

# Enter the foldername in your Drive where you have saved the unzipped
# assignment folder, e.g. 'cs231n/assignments/assignment1/'
FOLDERNAME = 'Coursework/ENPM703/assignment1'
assert FOLDERNAME is not None, "[!] Enter the foldername."

# Now that we've mounted your Drive, this ensures that
# the Python interpreter of the Colab VM can load
# python files from within it.
import sys
sys.path.append('/content/drive/My Drive/{}'.format(FOLDERNAME))

# This downloads the CIFAR-10 dataset to your Drive
# if it doesn't already exist.
%cd drive/My\ Drive/$FOLDERNAME/cs231n/datasets/
!bash get_datasets.sh
%cd /content/drive/My\ Drive/$FOLDERNAME

Mounted at /content/drive
/content/drive/My Drive/Coursework/ENPM703/assignment1/cs231n/datasets
/content/drive/My Drive/Coursework/ENPM703/assignment1


# Image features exercise
*Complete and hand in this completed worksheet (including its outputs and any supporting code outside of the worksheet) with your assignment submission. For more details see the [assignments page](http://vision.stanford.edu/teaching/cs231n/assignments.html) on the course website.*

We have seen that we can achieve reasonable performance on an image classification task by training a linear classifier on the pixels of the input image. In this exercise we will show that we can improve our classification performance by training linear classifiers not on raw pixels but on features that are computed from the raw pixels.

All of your work for this exercise will be done in this notebook.

In [2]:
import random
import numpy as np
from cs231n.data_utils import load_CIFAR10
import matplotlib.pyplot as plt


%matplotlib inline
plt.rcParams['figure.figsize'] = (10.0, 8.0) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

# for auto-reloading extenrnal modules
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

## Load data
Similar to previous exercises, we will load CIFAR-10 data from disk.

In [3]:
from cs231n.features import color_histogram_hsv, hog_feature

def get_CIFAR10_data(num_training=49000, num_validation=1000, num_test=1000):
    # Load the raw CIFAR-10 data
    cifar10_dir = 'cs231n/datasets/cifar-10-batches-py'

    # Cleaning up variables to prevent loading data multiple times (which may cause memory issue)
    try:
       del X_train, y_train
       del X_test, y_test
       print('Clear previously loaded data.')
    except:
       pass

    X_train, y_train, X_test, y_test = load_CIFAR10(cifar10_dir)

    # Subsample the data
    mask = list(range(num_training, num_training + num_validation))
    X_val = X_train[mask]
    y_val = y_train[mask]
    mask = list(range(num_training))
    X_train = X_train[mask]
    y_train = y_train[mask]
    mask = list(range(num_test))
    X_test = X_test[mask]
    y_test = y_test[mask]

    return X_train, y_train, X_val, y_val, X_test, y_test

X_train, y_train, X_val, y_val, X_test, y_test = get_CIFAR10_data()

## Extract Features
For each image we will compute a Histogram of Oriented
Gradients (HOG) as well as a color histogram using the hue channel in HSV
color space. We form our final feature vector for each image by concatenating
the HOG and color histogram feature vectors.

Roughly speaking, HOG should capture the texture of the image while ignoring
color information, and the color histogram represents the color of the input
image while ignoring texture. As a result, we expect that using both together
ought to work better than using either alone. Verifying this assumption would
be a good thing to try for your own interest.

The `hog_feature` and `color_histogram_hsv` functions both operate on a single
image and return a feature vector for that image. The extract_features
function takes a set of images and a list of feature functions and evaluates
each feature function on each image, storing the results in a matrix where
each column is the concatenation of all feature vectors for a single image.

In [9]:
from cs231n.features import *

num_color_bins = 15 # Number of bins in the color histogram
feature_fns = [hog_feature, lambda img: color_histogram_hsv(img, nbin=num_color_bins), lambda img: color_histogram_hsv(img, nbin=num_color_bins)]
X_train_feats = extract_features(X_train, feature_fns, verbose=True)
X_val_feats = extract_features(X_val, feature_fns)
X_test_feats = extract_features(X_test, feature_fns)

# Preprocessing: Subtract the mean feature
mean_feat = np.mean(X_train_feats, axis=0, keepdims=True)
X_train_feats -= mean_feat
X_val_feats -= mean_feat
X_test_feats -= mean_feat

# Preprocessing: Divide by standard deviation. This ensures that each feature
# has roughly the same scale.
std_feat = np.std(X_train_feats, axis=0, keepdims=True)
X_train_feats /= std_feat
X_val_feats /= std_feat
X_test_feats /= std_feat

# Preprocessing: Add a bias dimension
X_train_feats = np.hstack([X_train_feats, np.ones((X_train_feats.shape[0], 1))])
X_val_feats = np.hstack([X_val_feats, np.ones((X_val_feats.shape[0], 1))])
X_test_feats = np.hstack([X_test_feats, np.ones((X_test_feats.shape[0], 1))])

Done extracting features for 1000 / 49000 images
Done extracting features for 2000 / 49000 images
Done extracting features for 3000 / 49000 images
Done extracting features for 4000 / 49000 images
Done extracting features for 5000 / 49000 images
Done extracting features for 6000 / 49000 images
Done extracting features for 7000 / 49000 images
Done extracting features for 8000 / 49000 images
Done extracting features for 9000 / 49000 images
Done extracting features for 10000 / 49000 images
Done extracting features for 11000 / 49000 images
Done extracting features for 12000 / 49000 images
Done extracting features for 13000 / 49000 images
Done extracting features for 14000 / 49000 images
Done extracting features for 15000 / 49000 images
Done extracting features for 16000 / 49000 images
Done extracting features for 17000 / 49000 images
Done extracting features for 18000 / 49000 images
Done extracting features for 19000 / 49000 images
Done extracting features for 20000 / 49000 images
Done extr

## Train SVM on features
Using the multiclass SVM code developed earlier in the assignment, train SVMs on top of the features extracted above; this should achieve better results than training SVMs directly on top of raw pixels.

In [10]:
# Use the validation set to tune the learning rate and regularization strength

from cs231n.classifiers.linear_classifier import LinearSVM

learning_rates = [1e-9, 1e-8, 1e-7]
regularization_strengths = [5e4, 5e5, 5e6]

results = {}
best_val = -1
best_svm = None

################################################################################
# TODO:                                                                        #
# Use the validation set to set the learning rate and regularization strength. #
# This should be identical to the validation that you did for the SVM; save    #
# the best trained classifer in best_svm. You might also want to play          #
# with different numbers of bins in the color histogram. If you are careful    #
# you should be able to get accuracy of near 0.44 on the validation set.       #
################################################################################
# *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****

# pass
for lr in learning_rates:
    for rs in regularization_strengths:
        svm = LinearSVM()
        svm.train(X_train_feats, y_train, learning_rate=lr, reg=rs, num_iters=3000,
                  verbose=False)
        y_train_pred = svm.predict(X_train_feats)
        train_acc = np.mean(y_train == y_train_pred)
        y_val_pred = svm.predict(X_val_feats)
        val_acc = np.mean(y_val == y_val_pred)

        results[lr, rs] = [train_acc, val_acc]
        if val_acc > best_val:
            best_val = val_acc
            best_svm = svm

# *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****

# Print out results.
for lr, reg in sorted(results):
    train_accuracy, val_accuracy = results[(lr, reg)]
    print('lr %e reg %e train accuracy: %f val accuracy: %f' % (
                lr, reg, train_accuracy, val_accuracy))

print('best validation accuracy achieved: %f' % best_val)

lr 1.000000e-09 reg 5.000000e+04 train accuracy: 0.110041 val accuracy: 0.103000
lr 1.000000e-09 reg 5.000000e+05 train accuracy: 0.096918 val accuracy: 0.095000
lr 1.000000e-09 reg 5.000000e+06 train accuracy: 0.422449 val accuracy: 0.426000
lr 1.000000e-08 reg 5.000000e+04 train accuracy: 0.151755 val accuracy: 0.156000
lr 1.000000e-08 reg 5.000000e+05 train accuracy: 0.423592 val accuracy: 0.437000
lr 1.000000e-08 reg 5.000000e+06 train accuracy: 0.418082 val accuracy: 0.414000
lr 1.000000e-07 reg 5.000000e+04 train accuracy: 0.423082 val accuracy: 0.435000
lr 1.000000e-07 reg 5.000000e+05 train accuracy: 0.419735 val accuracy: 0.414000
lr 1.000000e-07 reg 5.000000e+06 train accuracy: 0.340122 val accuracy: 0.343000
best validation accuracy achieved: 0.437000


In [11]:
# Evaluate your trained SVM on the test set: you should be able to get at least 0.40
y_test_pred = best_svm.predict(X_test_feats)
test_accuracy = np.mean(y_test == y_test_pred)
print(test_accuracy)

0.419


In [12]:
# An important way to gain intuition about how an algorithm works is to
# visualize the mistakes that it makes. In this visualization, we show examples
# of images that are misclassified by our current system. The first column
# shows images that our system labeled as "plane" but whose true label is
# something other than "plane".

examples_per_class = 8
classes = ['plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']
for cls, cls_name in enumerate(classes):
    idxs = np.where((y_test != cls) & (y_test_pred == cls))[0]
    idxs = np.random.choice(idxs, examples_per_class, replace=False)
    for i, idx in enumerate(idxs):
        plt.subplot(examples_per_class, len(classes), i * len(classes) + cls + 1)
        plt.imshow(X_test[idx].astype('uint8'))
        plt.axis('off')
        if i == 0:
            plt.title(cls_name)
plt.show()

### Inline question 1:
Describe the misclassification results that you see. Do they make sense?


$\color{blue}{\textit Your Answer:}$

Upon extracting texture and color data, photos with similar forms or colors could be mistakenly labeled. Animal classes, for instance, all have brown coloring, a tail, and four feet. The background colors—white and blue seem to have a greater influence on the type of plane or ship than the shape. However, there are misclassified photos with no evident shared qualities in certain classes, such as frog, truck, and automobile. This could indicate that insufficient features were extracted by the HOG and color histogram methods to properly categorize the pictures.



## Neural Network on image features
Earlier in this assigment we saw that training a two-layer neural network on raw pixels achieved better classification performance than linear classifiers on raw pixels. In this notebook we have seen that linear classifiers on image features outperform linear classifiers on raw pixels.

For completeness, we should also try training a neural network on image features. This approach should outperform all previous approaches: you should easily be able to achieve over 55% classification accuracy on the test set; our best model achieves about 60% classification accuracy.

In [13]:
# Preprocessing: Remove the bias dimension
# Make sure to run this cell only ONCE
print(X_train_feats.shape)
X_train_feats = X_train_feats[:, :-1]
X_val_feats = X_val_feats[:, :-1]
X_test_feats = X_test_feats[:, :-1]

print(X_train_feats.shape)

(49000, 175)
(49000, 174)


In [28]:
data = {
    "X_train" : X_train_feats,
    "X_val" : X_val_feats,
    "X_test" : X_test_feats,
    "y_train" : y_train,
    "y_val" : y_val,
    "y_test" : y_test
}

In [33]:
from cs231n.classifiers.fc_net import TwoLayerNet
from cs231n.solver import Solver
from itertools import product

# Input dimensions and other parameters
input_dim = X_train_feats.shape[1]
hidden_dim = 500
num_classes = 10

################################################################################
# TODO: Train a two-layer neural network on image features. You may want to    #
# cross-validate various parameters as in previous sections. Store your best   #
# model in the best_net variable.                                              #
################################################################################
# *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****

# Initialize variables to store the best model and its parameters
best_net = None
best_accuracy = 0.0
best_params = None

# Hyperparameter ranges
learning_rates = [0.1, 0.105]
regularization_strengths = [2e-5, 3e-5]
lr_decay = [0.9, 1.0]

# Create all combinations of hyperparameters
grids = list(product(learning_rates, regularization_strengths, lr_decay))

# Loop through each combination of hyperparameters
total_combinations = len(grids)
for idx, (lr, reg, dec) in enumerate(grids):
    print(f'Starting training with lr={lr}, reg={reg}, decay={dec} (Combination {idx+1}/{total_combinations})')

    # Create a new instance of the TwoLayerNet for each parameter set
    net = TwoLayerNet(input_dim=input_dim,
                      hidden_dim=hidden_dim,
                      num_classes=num_classes,
                      weight_scale=1e-3,
                      reg=reg)

    # Create a new solver for this combination
    solver = Solver(net, data,
                    update_rule='sgd',
                    optim_config={'learning_rate': lr},
                    lr_decay=dec,
                    num_epochs=14,  # Set the number of epochs to train
                    batch_size=200,
                    print_every=100)

    # Train the model
    solver.train()

    print(f'Finished training with lr={lr}, reg={reg}, decay={dec}\n')

    # Check if this is the best accuracy so far
    final_val_acc = solver.val_acc_history[-1]
    if final_val_acc > best_accuracy:
        best_params = (lr, reg, dec)
        best_net = net
        best_accuracy = final_val_acc

# Print the best results
print(f'Best accuracy: {best_accuracy}')
print('Best params:', best_params)

# *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****

Starting training with lr=0.1, reg=2e-05, decay=0.9 (Combination 1/8)
(Iteration 1 / 3430) loss: 2.302614
(Epoch 0 / 14) train acc: 0.087000; val_acc: 0.107000
(Iteration 101 / 3430) loss: 2.193658
(Iteration 201 / 3430) loss: 1.762734
(Epoch 1 / 14) train acc: 0.445000; val_acc: 0.411000
(Iteration 301 / 3430) loss: 1.467168
(Iteration 401 / 3430) loss: 1.460248
(Epoch 2 / 14) train acc: 0.510000; val_acc: 0.507000
(Iteration 501 / 3430) loss: 1.327508
(Iteration 601 / 3430) loss: 1.295423
(Iteration 701 / 3430) loss: 1.390969
(Epoch 3 / 14) train acc: 0.536000; val_acc: 0.510000
(Iteration 801 / 3430) loss: 1.154440
(Iteration 901 / 3430) loss: 1.282816
(Epoch 4 / 14) train acc: 0.523000; val_acc: 0.514000
(Iteration 1001 / 3430) loss: 1.225342
(Iteration 1101 / 3430) loss: 1.321424
(Iteration 1201 / 3430) loss: 1.305530
(Epoch 5 / 14) train acc: 0.533000; val_acc: 0.525000
(Iteration 1301 / 3430) loss: 1.218100
(Iteration 1401 / 3430) loss: 1.322140
(Epoch 6 / 14) train acc: 0.55000

In [34]:
# Run your best neural net classifier on the test set. You should be able
# to get more than 55% accuracy.

y_test_pred = np.argmax(solver.model.loss(data['X_test']), axis=1)
test_acc = (y_test_pred == data['y_test']).mean()
print(test_acc)

0.565
